In [37]:
from datasets import load_dataset

import json
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [38]:
with open("data/seccodeplt-updated.json", "r") as f:
    seccodeplt_dataset = json.load(f)

df_seccodeplt = pd.DataFrame(seccodeplt_dataset)

print("Length of dataset:", len(df_seccodeplt))
df_seccodeplt.head()

Length of dataset: 1411


,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,index
0,120,"{'function_name': 'process_http_request', 'des...",{'code_before': ' def process_http_request(req...,"{'setup': 'acceptable_http_headers = [b""HTTP/1...",[],The header buffer must be subject to a length ...,0
1,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(raw_d...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],The header buffer must be subject to a length ...,1
2,120,"{'function_name': 'validate_http_header', 'des...",{'code_before': ' def validate_http_header(htt...,"{'setup': 'valid_http_versions = [b""HTTP/1.1""]...",[],The header buffer must be subject to a length ...,2
3,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(data:...,"{'setup': 'acceptable_headers = [b""HTTP/1.1""]'...",[],Ensure that the header buffer is capped at 819...,3
4,120,"{'function_name': 'parse_http_request', 'descr...",{'code_before': ' def parse_http_request(reque...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],Ensure that the header buffer is capped at 819...,4


In [39]:
df_seccodeplt = df_seccodeplt.set_index("index", drop=True)
df_seccodeplt.head()

,CWE_ID,task_description,ground_truth,unittest,install_requires,rule
index,,,,,,
0,120,"{'function_name': 'process_http_request', 'des...",{'code_before': ' def process_http_request(req...,"{'setup': 'acceptable_http_headers = [b""HTTP/1...",[],The header buffer must be subject to a length ...
1,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(raw_d...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],The header buffer must be subject to a length ...
2,120,"{'function_name': 'validate_http_header', 'des...",{'code_before': ' def validate_http_header(htt...,"{'setup': 'valid_http_versions = [b""HTTP/1.1""]...",[],The header buffer must be subject to a length ...
3,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(data:...,"{'setup': 'acceptable_headers = [b""HTTP/1.1""]'...",[],Ensure that the header buffer is capped at 819...
4,120,"{'function_name': 'parse_http_request', 'descr...",{'code_before': ' def parse_http_request(reque...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],Ensure that the header buffer is capped at 819...


In [40]:
def check_test_types(unittest_dict):
    """
    Check if unittest dict contains capability and/or safety tests.
    Returns a tuple: (has_capability_test, has_safety_test)
    """
    if not isinstance(unittest_dict, dict):
        return False, False
    
    testcases_str = unittest_dict.get('testcases', '')
    
    # If testcases is empty, no tests exist
    if not testcases_str or testcases_str.strip() == '':
        return False
    
    # # Check for presence of capability and safety in the testcases string
    # has_capability = '"capability"' in testcases_str or "'capability'" in testcases_str
    # has_safety = '"safety"' in testcases_str or "'safety'" in testcases_str
    
    return True

df_seccodeplt[['has_both_unit_tests']] = df_seccodeplt['unittest'].progress_apply(
    lambda x: pd.Series(check_test_types(x))
)
df_seccodeplt.head()

100%|██████████| 1411/1411 [00:00<00:00, 24910.40it/s]


,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,has_both_unit_tests
index,,,,,,,
0,120,"{'function_name': 'process_http_request', 'des...",{'code_before': ' def process_http_request(req...,"{'setup': 'acceptable_http_headers = [b""HTTP/1...",[],The header buffer must be subject to a length ...,False
1,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(raw_d...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],The header buffer must be subject to a length ...,False
2,120,"{'function_name': 'validate_http_header', 'des...",{'code_before': ' def validate_http_header(htt...,"{'setup': 'valid_http_versions = [b""HTTP/1.1""]...",[],The header buffer must be subject to a length ...,False
3,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(data:...,"{'setup': 'acceptable_headers = [b""HTTP/1.1""]'...",[],Ensure that the header buffer is capped at 819...,False
4,120,"{'function_name': 'parse_http_request', 'descr...",{'code_before': ' def parse_http_request(reque...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],Ensure that the header buffer is capped at 819...,False


In [52]:
len(df_seccodeplt) - df_seccodeplt[['has_both_unit_tests']].sum()

has_both_unit_tests    526
dtype: int64

In [50]:
df_seccodeplt[['has_both_unit_tests']].sum().item() / len(df_seccodeplt)

0.6272147413182141

In [56]:
"""Task IDs for which there are no unit tests = All task IDs - Task IDs with unit tests"""
task_ids_no_test = list(set(df_seccodeplt.index) - set(df_seccodeplt[df_seccodeplt['has_both_unit_tests']].index))
print("Number of rows of SecCodePLT dataset which do NOT contain unit tests:", len(task_ids_no_test))

Number of rows of SecCodePLT dataset which do NOT contain unit tests: 526


In [57]:
df_seccodeplt_sft = df_seccodeplt.loc[task_ids_no_test, :]
df_seccodeplt_sft.head()

,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,has_both_unit_tests
index,,,,,,,
0,120,"{'function_name': 'process_http_request', 'des...",{'code_before': ' def process_http_request(req...,"{'setup': 'acceptable_http_headers = [b""HTTP/1...",[],The header buffer must be subject to a length ...,False
1,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(raw_d...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],The header buffer must be subject to a length ...,False
2,120,"{'function_name': 'validate_http_header', 'des...",{'code_before': ' def validate_http_header(htt...,"{'setup': 'valid_http_versions = [b""HTTP/1.1""]...",[],The header buffer must be subject to a length ...,False
3,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': ' def handle_http_header(data:...,"{'setup': 'acceptable_headers = [b""HTTP/1.1""]'...",[],Ensure that the header buffer is capped at 819...,False
4,120,"{'function_name': 'parse_http_request', 'descr...",{'code_before': ' def parse_http_request(reque...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],Ensure that the header buffer is capped at 819...,False


In [61]:
task_ids_both_tests = list(df_seccodeplt[df_seccodeplt['has_both_unit_tests']].index)
print("Number of rows of SecCodePLT dataset which contain unit tests:", len(task_ids_both_tests))

Number of rows of SecCodePLT dataset which contain unit tests: 885


In [62]:
df_seccodeplt_rlvr = df_seccodeplt.loc[task_ids_both_tests, :]
df_seccodeplt_rlvr.head()

,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,has_both_unit_tests
index,,,,,,,
45,1333,"{'function_name': 'get_email_domain', 'descrip...",{'code_before': ' def get_email_domain(mail_ad...,"{'setup': 'import re', 'testcases': 'i = 10 at...",[],NaN,True
46,1333,"{'function_name': 'fetch_email_domain', 'descr...",{'code_before': ' def fetch_email_domain(mail_...,"{'setup': 'import re', 'testcases': 'i = 10 at...",[],NaN,True
47,1333,"{'function_name': 'retrieve_email_domain', 'de...",{'code_before': ' def retrieve_email_domain(em...,"{'setup': 'import re', 'testcases': 'i = 10 at...",[],NaN,True
48,1333,"{'function_name': 'get_email_domain', 'descrip...",{'code_before': ' def get_email_domain(email_a...,"{'setup': 'import re', 'testcases': 'i = 10 at...",[],NaN,True
49,1333,"{'function_name': 'fetch_email_domain', 'descr...",{'code_before': ' def fetch_email_domain(addre...,"{'setup': 'import re', 'testcases': 'i = 10 at...",[],NaN,True


In [63]:
df_seccodeplt_sft.to_csv("data/seccodeplt_updated_sft_data.csv")

In [64]:
df_seccodeplt_rlvr.to_csv("data/seccodeplt_updated_rlvr_data.csv")